In [7]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
import random
from datetime import datetime
import string
from hashlib import sha1

In [8]:
def getDF(name):
  df = pd.read_excel(name)
  return df

In [9]:
def getData(dF, ind):
  games = dF['te_Games'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  cities = dF['te_Cities'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  sports = dF['te_Sports'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  medals = dF['te_Medals'][ind].replace('[', '').replace(']', '').replace("'", '').split(', ')
  events = dF['te_Events'][ind].replace('[', '').replace(']', '').split('", "')
  events = [x.replace('"', '') for x in events]
  categories = [
                '[[వర్గం:తెవికీ ఒలింపిక్ అథ్లెట్]]\n',
                '[[వర్గం:' + dF['te_Country'][ind] +' అథ్లెట్]]\n',
                '[[వర్గం:' + sports[0] +' అథ్లెట్]]\n'
  ]
  image = dF['Thumbnail'][ind]
  if image == 'None':
    image = ''
  else:
    image = dF['Thumbnail'][ind].split('/')[-1]
    if '?' in image:
      image = image[:image.index('?')]
  caption = dF['te_Caption'][ind]
  if caption == 'None':
    caption = ''
  weight = str(dF['Weight'][ind]).strip()
  if weight == 'nan':
    weight = 'None'
  return {
        'name': str(dF['te_Name'][ind]).strip(),
        'image': str(image).strip(),
        'caption': str(caption).strip(),
        'Gender': str(dF['Gender'][ind]).strip(),
        'dob': str(dF['Date_Of_Birth'][ind]).strip(),
        'pob': str(dF['te_Birth Place'][ind]).strip(),
        'height': str(dF['Height'][ind]).strip(),
        'weight': weight,
        'country': str(dF['te_Country'][ind]).strip(),
        'NOC': str(dF['NOC'][ind]).strip(),
        'sport': str(sports[0]).strip(),
        'games': games,
        'cities': cities,
        'sports': sports,
        'events': events,
        'medals': medals,
        'stateOfOrigin': str(dF['te_StateOfOrigin'][ind]).strip(),
        'residence': str(dF['te_Residence'][ind]).strip(),
        'education': str(dF['te_Education'][ind]).strip(),
        'coach': str(dF['te_Coach'][ind]).strip(),
        'club': str(dF['te_Club'][ind]).strip(),
        'nationality': str(dF['te_Country'][ind]).strip(),
        'categories': categories,
        'dob_ref': "{{Cite web|url=https://dbpedia.org/page/" + dF['Wiki_Links'][ind].split('/')[-1] + "|title= Dbpedia Page of " + dF['Name'][ind] + "|language=en}}"
    }

In [10]:
def getProfessionalData(data):
  if data['Gender'] == 'M':
    v1 = 'పాల్గొన్నాడు'
    v2 = 'గెలుచుకున్నాడు'
  else:
    v1 = 'పాల్గొంది'
    v2 = 'గెలుచుకుంది'
  pathakam = " [https://te.wikipedia.org/wiki/పతకం పతకం]"
  professionalLifeData = []
  i = 0
  table = '''
  {| class="wikitable"
|+
!గేమ్స్ 
!నగరం
!ఈవెంట్ 
!మెడల్ 
|-
'''
  while i < len(data['games']):
    indexs = []
    for j in range(i, len(data['games'])):
      if data['games'][j] == data['games'][i]:
        indexs.append(j)
    if indexs[-1] == 0:
      i += 1
    else:
      i = indexs[-1] + 1
    lst = [
      data['name'] + ' ' + data['games'][indexs[0]].split(' ')[0] + ' '  + "[https://te.wikipedia.org/wiki" + data['cities'][indexs[0]].replace(' ',  '_') + " " + data['cities'][indexs[0]] + "]" + " నగరంలో నిర్వహించబడిన " + data['games'][indexs[0]].split(' ')[1] + ' ఒలింపిక్స్లో' +  ' ' + data['sports'][indexs[0]] + " డిసిప్లిన్ లో ",
      "[https://te.wikipedia.org/wiki" + data['cities'][indexs[0]].replace(' ',  '_') + " " + data['cities'][indexs[0]] + "]" + " నగరంలో నిర్వహించబడిన " + data['games'][indexs[0]] + " ఒలింపిక్స్లో " +  data['name'] + ' ' + data['sports'][indexs[0]] + " డిసిప్లిన్ లో "
      ]
    table += '| rowspan="'+str(len(indexs))+ '" | ' + data['games'][indexs[0]] + " ఒలింపిక్స్" + "\n"
    table += '| rowspan="'+str(len(indexs))+ '" | ' + data['cities'][indexs[0]] + "\n"
    info = lst[random.choice([x for x in range(100)]) % 2]
    for m in indexs:
      if indexs[0] == m:
        info += data['events'][m]
      else:
        info += ', ' + data['events'][m]
      table += '| ' + data['events'][m] + '\n'
      table += "| " + data['medals'][m] + '\n'
      table += '|-\n'
    if len(indexs) == 1:
      info += " ఈవెంట్లో పాల్గొనగా "
    else:
      info += " ఈవెంట్లలో పాల్గొనగా "
    checkIfMedal = 1
    for m in indexs:
      if(data['medals'][m] != 'nan'):
        checkIfMedal = 0
        info += ', ' + data['events'][m] + " లో " + data['medals'][m] + pathakam
        pathakam = " పతకం"
    if checkIfMedal:      
      info = info[:len(info) - 11] + v1 + "."
    else:
      info += ' ' + v2 + '.'
    if data['games'][indexs[0]].split(' ')[0] != '2012' and data['games'][indexs[0]].split(' ')[0] != '2016':
      professionalLifeData.append("=== [https://te.wikipedia.org/wiki/" + data['games'][indexs[0]].split(' ')[0] + "_వేసవి_ఒలింపిక్_క్రీడలు " + data['games'][indexs[0]].split(' ')[0] + " ఒలింపిక్స్] " + "===")
    else:
      professionalLifeData.append("=== " + data['games'][indexs[0]].split(' ')[0] + " ఒలింపిక్స్ ===")
    professionalLifeData.append(info)
  table = table[:-2] + "}"
  table = table.replace("nan", '-')
  data['professionalData'] = table
  return data

In [11]:
def getMedalsData(data):
  if len(data['medals']) != data['medals'].count('nan'):
    medalsData = ["| medaltemplates = "]
    for i in range(len(data['medals'])):
      if data['medals'][i] == 'గోల్డ్':
        medalsData.append("{{Medal" + "Gold | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
      elif data['medals'][i] == 'సిల్వర్':
        medalsData.append("{{Medal" + "Silver | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
      elif data['medals'][i] == 'బ్రోన్జ్' or data['medals'][i] == 'బ్రాంజ్':
        medalsData.append("{{Medal" + "Bronze | " + data['games'][i].split(' ')[0] + " " + data['cities'][i] + " | " + data['events'][i] + "}}")
    # medalsData.append("}}")
    medalsData = '\n'.join(medalsData)
    data['medalsData'] = medalsData
  return data

In [12]:
tewiki = '''<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.10/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.10/ http://www.mediawiki.org/xml/export-0.10.xsd" version="0.10" xml:lang="te">
	<siteinfo>
		<sitename>tewiki</sitename>
		<dbname>indicwiki</dbname>
		<base>https://tewiki.iiit.ac.in/index.php/%E0%B0%AE%E0%B1%8A%E0%B0%A6%E0%B0%9F%E0%B0%BF_%E0%B0%AA%E0%B1%87%E0%B0%9C%E0%B1%80</base>
		<generator>MediaWiki 1.34.0</generator>
		<case>first-letter</case>
		<namespaces>
			<namespace key="-2" case="first-letter">మీడియా</namespace>
			<namespace key="-1" case="first-letter">ప్రత్యేక</namespace>
			<namespace key="0" case="first-letter" />
			<namespace key="1" case="first-letter">చర్చ</namespace>
			<namespace key="2" case="first-letter">వాడుకరి</namespace>
			<namespace key="3" case="first-letter">వాడుకరి చర్చ</namespace>
			<namespace key="4" case="first-letter">Project</namespace>
			<namespace key="5" case="first-letter">Project చర్చ</namespace>
			<namespace key="6" case="first-letter">దస్త్రం</namespace>
			<namespace key="7" case="first-letter">దస్త్రంపై చర్చ</namespace>
			<namespace key="8" case="first-letter">మీడియావికీ</namespace>
			<namespace key="9" case="first-letter">మీడియావికీ చర్చ</namespace>
			<namespace key="10" case="first-letter">మూస</namespace>
			<namespace key="11" case="first-letter">మూస చర్చ</namespace>
			<namespace key="12" case="first-letter">సహాయం</namespace>
			<namespace key="13" case="first-letter">సహాయం చర్చ</namespace>
			<namespace key="14" case="first-letter">వర్గం</namespace>
			<namespace key="15" case="first-letter">వర్గం చర్చ</namespace>
			<namespace key="828" case="first-letter">మాడ్యూల్</namespace>
			<namespace key="829" case="first-letter">మాడ్యూల్ చర్చ</namespace>
			<namespace key="2300" case="first-letter">Gadget</namespace>
			<namespace key="2301" case="first-letter">Gadget talk</namespace>
			<namespace key="2302" case="case-sensitive">Gadget definition</namespace>
			<namespace key="2303" case="case-sensitive">Gadget definition talk</namespace>
			<namespace key="2600" case="first-letter">Topic</namespace>
		</namespaces>
	</siteinfo>\n'''

In [13]:
user_id = 18884
username = 'SantoshJonnakuti'
def sha36(page_id):
	page_id = str(page_id).encode('utf-8')
	sha16 =sha1(page_id).hexdigest()
	sha10 =int(sha16, 16)

	chars =[]
	alphabets = string.digits +string.ascii_lowercase
	while sha10>0:
		sha10, r = divmod(sha10, 36)
		chars.append(alphabets[r])
	
	return ''.join(reversed(chars))


def writePage(title, wikiText, fobj, page_id):
	global user_id, username

	pglen = len(wikiText)
	time =datetime.now().strftime("%Y-%m-%dT%H-%M-%SZ")
	if (" 00:00:00") in wikiText:
		wikiText = wikiText.replace(" 00:00:00", '')
	curPage ='''\n\n
	<page>
		<title>''' +title +'''</title>
		<ns>0</ns>
		<id>''' +str(page_id) +'''</id>
		<revision>
			<id>''' +str(page_id) +'''</id>
			<timestamp>'''+time+'''</timestamp>
			<contributor>
				<username>''' +username +'''</username>
				<id>''' +str(user_id) +'''</id>
			</contributor>
			<comment>xmlpage created</comment>
			<model>wikitext</model>
			<format>text/x-wiki</format>
			<text xml:space="preserve" bytes="''' +str(pglen) +'''">
			\n''' +wikiText +'''
			</text>
			<sha1>''' +sha36(page_id) +'''</sha1>
		</revision>
	</page>
	\n\n'''
	fobj.write(curPage)
	return

In [14]:
fileLoader = FileSystemLoader('/')
env = Environment(loader=fileLoader)
env.globals.update(zip=zip)
template = env.get_template('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/template.j2')

In [15]:
def genXML(name, i, page_id):
  count = 0
  wCount = 0
  dCount = 0
  iCount = 0
  XMLFileObj = open('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/XML/Sheet-' + i +'.xml', 'w')
  XMLFileObj.write(tewiki+'\n')
  XMLFileObj.close()
  df = getDF(name)
  for ind in df.index:
    # print(ind)
    try:
      data = getData(df, ind)
      data = getProfessionalData(data)
      data = getMedalsData(data)
      text = template.render(data)
      title = data['name']
      text = text.strip()
      XMLFileObj = open('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/XML/Sheet-' + i +'.xml', 'a')
      writePage(title, text, XMLFileObj, page_id)
      XMLFileObj.close()
      count += 1
      page_id += 1
    except Exception as e:
      # print(e)
      if 'datetime' in str(e):
        dCount += 1
      elif 'list index' in str(e):
        iCount += 1
      else:
        wCount += 1

  XMLFileObj = open('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/XML/Sheet-' + i +'.xml', 'a')
  XMLFileObj.write('</mediawiki>')
  XMLFileObj.close()
  print(i, count, wCount, dCount, iCount)
  return page_id

In [16]:
page_id = 795001
for i in range(1, 32):
  page_id = genXML('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/olympic games Final Data/olympic games DS/Olympicgames sheet-'+ str(i) +'.xlsx', str(i), page_id)
  if i == 11:
    page_id = genXML('/content/drive/MyDrive/IndicWiki-OlympicGames/Athletes-Data/olympic games Final Data/olympic games DS/Olympicgames sheet-'+ str('11(2)') +'.xlsx', str('11(2)'), page_id)

1 975 0 0 26
2 967 0 0 33
3 961 0 0 39
4 933 0 0 67
5 956 0 0 44
6 956 0 0 44
7 934 0 0 66
8 972 0 0 28
9 941 0 0 59
10 979 0 0 21
11 944 0 0 56
11(2) 954 0 0 46
12 958 0 0 42
13 974 0 0 26
14 980 0 0 20
15 960 0 0 40
16 971 0 0 29
17 972 0 0 28
18 946 0 0 54
19 969 0 0 31
20 968 0 0 32
21 958 0 0 42
22 973 0 0 27
23 947 0 0 53
24 968 0 0 32
25 971 0 0 29
26 974 0 0 26
27 956 0 0 44
28 944 0 0 56
29 976 0 0 24
30 942 1 0 57
31 650 0 0 30
